In [1]:
import logging
import pdb
logging.basicConfig(level=logging.INFO)
%matplotlib inline
import pyfiglet
import numpy as np
# np.seterr(all='raise')
import os

from local_repo import *
from repos import *
from metrics import *
from analysis import *


repos = [
    "ErikBrendel/LudumDare", # 8 minutes BTM
    "eclipse/eclipse.jdt.core",  # from duerschmidt
    "jenkinsci/jenkins",
    "wumpz/jhotdraw",  # TODO instead use wrandelshofer/jhotdraw and sub-directories as projects from there
    "neuland/jade4j",
    "jfree/jfreechart",
    "brettwooldridge/HikariCP",  # 1.2h BTM
    "adamfisk/LittleProxy",  # 1.5h BTM
    "dynjs/dynjs",  # 2.5h BTM
    # "SonarSource/sonarqube",  # quite big / prob. several hours BTM
    "square/okhttp",
    "eclipse/che",
    "elastic/elasticsearch",
    "apache/camel",
    "jOOQ/jOOQ",
]
repos = [
    #"wrandelshofer/jhotdraw/JHotDraw",
    #"wrandelshofer/jhotdraw/jhotdraw6",
    #"wrandelshofer/jhotdraw/jhotdraw7",
    #"wrandelshofer/jhotdraw/jhotdraw8",
   "ErikBrendel/LudumDare", 
]



for repo in repos:
    r = LocalRepo(repo)
    print(pyfiglet.figlet_format(r.name))
    r.update()
    
    for view in ["structural", "evolutionary", "linguistic"]: # ["structural", "evolutionary", "linguistic"]
        # MetricManager.clear(r, view)

        coupling_graph = MetricManager.get(r, view)
        # coupling_graph.plaintext_save(repo)
        # coupling_graph.html_save(repo)
        

        print("\nResults from the " + view + " view:\n")

        coupling_graph.print_statistics()
        # coupling_graph.print_most_linked_nodes()
        # coupling_graph.show_weight_histogram()
        # coupling_graph.visualize(use_spring=False, with_labels=False)
        # coupling_graph.visualize(use_spring=True, with_labels=False)
        print("\n")
    # continue
    
    # amalysis
    repo_tree = r.get_tree()
    def node_filter(node_path):
        tree_node = repo_tree.find_node(node_path)
        return tree_node.get_type() == "method" and tree_node.get_line_span() >= 2
    def node_pair_filter(node_a_path, node_b_path):
        return True # path_module_distance(node_a_path, node_b_path) >= 6
    
    
    a = lambda : analyze_disagreements(r, ["structural", "evolutionary", "linguistic", "module_distance"], [
        [0,    1,    1,    1, "Non-struct?"],
        [1,    1,    1,    0, "Too far apart"],
        [0,    0,    1,    0, "Independent Feature duplication"],
        [0,    1,    1,    0, "Parallel-Maintained Feature duplication"],
        [1, None,    0,    1, "Weakly modularized code"],
        [0,    0,    0,    1, "Close but totally unrelated"],
    ], node_filter, node_pair_filter)
    # a()
    %prun -s cumulative a()
    
print("\nProgram is over!")

# https://arrow.apache.org/docs/python/plasma.html#putting-and-getting-python-objects

/usr/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


 _____      _ _    ____                     _      _    ___              _ 
| ____|_ __(_) | _| __ ) _ __ ___ _ __   __| | ___| |  / / |   _   _  __| |
|  _| | '__| | |/ /  _ \| '__/ _ \ '_ \ / _` |/ _ \ | / /| |  | | | |/ _` |
| |___| |  | |   <| |_) | | |  __/ | | | (_| |  __/ |/ / | |__| |_| | (_| |
|_____|_|  |_|_|\_\____/|_|  \___|_| |_|\__,_|\___|_/_/  |_____\__,_|\__,_|
                                                                           
                 ____                 
 _   _ _ __ ___ |  _ \  __ _ _ __ ___ 
| | | | '_ ` _ \| | | |/ _` | '__/ _ \
| |_| | | | | | | |_| | (_| | | |  __/
 \__,_|_| |_| |_|____/ \__,_|_|  \___|
                                      

updating ErikBrendel/LudumDare...
Repo is up to date!
No precalculated structural values found, starting calculations...
Analyzing 44 of 44 files, the rest was skipped as tests or samples
Skipped 0 test and sample roots: []


Discovering classes:   0%|          | 0/44 [00:00<?, ?it/s]

Analyzing 44 of 44 files, the rest was skipped as tests or samples
Skipped 0 test and sample roots: []
Found 565 directories, files, classes, methods and fields!


Building Import Graph 2:   0%|          | 0/44 [00:00<?, ?it/s]

...

Extracting language corpus:   0%|          | 0/44 [00:00<?, ?it/s]

Amount of documents: 436
Total Amount of words: 9311
Vocab size: 502


BTM Gibbs Sampling:   0%|          | 0/1000 [00:00<?, ?it/s]

[BTM] Parallel Biterm Topic Modeling:
[BTM]   Iterations: 1000
[BTM]   Maximum vocabulary size: 4000
[BTM]   Number of Topics: 10
[BTM]   Alpha: 1
[BTM]   Beta: 0.01
[BTM]   Worker threads: 127
[BTM]   Number of top words to show: 10
[BTM]   Number of top documents to show: 5
[BTM]   Number of documents: 436
[BTM] Total amount of words: 707, vocab size: 707
[BTM] Total amount of biterms: 52151
[BTM] 
[BTM] Top documents per topic:
[BTM] Topic 0: score option view text show game box screen username high
[BTM] ->Coherence: -67.4698
[BTM]    text box view text box view text text 20 550 1560 330 font font monospaced font plain 60
[BTM]    game screen draw graphics2 d g wheel rotation mouse wheel rotation time visible 2 g color color 5...
[BTM]    game screen high score available
[BTM]    option username system property user name replace replace replace replace
[BTM]    game screen highscore
[BTM] 
[BTM] Topic 1: manager sound laser play y player update x frame time
[BTM] ->Coherence: -63.1

NameError: name 'np' is not defined